In [ ]:
import requests
import sqlite3
import datetime
import time
import os

# === CONFIGURATION ===
REALTIME_API_URL = "https://data.explore.star.fr/api/records/1.0/search/"
STOP_API_URL = "https://data.explore.star.fr/api/records/1.0/search/"
REALTIME_DATASET = "tco-bus-circulation-passages-tr"
STOP_DATASET = "tco-bus-topologie-pointsarret"
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
DB_PATH = os.path.join(BASE_DIR, "collecte_donnees", "urban_mobility.db")

# === INIT BASE DE DONNÉES ===
def init_db():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Table des arrêts
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS star_stops (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        latitude REAL,
        longitude REAL
    )
    """)

    # Table des passages temps réel
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS star_realtime (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        timestamp TEXT,
        line TEXT,
        destination TEXT,
        stop_name TEXT,
        latitude REAL,
        longitude REAL,
        status TEXT,
        arrival_scheduled TEXT,
        arrival_real TEXT
    )
    """)

    conn.commit()
    conn.close()
    print("✅ Base initialisée.\n")

# === COLLECTE DES ARRETS AVEC COORDS ===
def get_star_stops():
    print("🌐 Collecte des arrêts STAR avec coordonnées...")
    try:
        response = requests.get(STOP_API_URL, params={
            "dataset": STOP_DATASET,
            "rows": 1000
        })
        data = response.json()
        records = data.get("records", [])
        return [
            {
                "name": rec["fields"].get("nom", "").strip(),
                "lat": rec["fields"]["coordonnees"][0],
                "lon": rec["fields"]["coordonnees"][1],
            }
            for rec in records if "coordonnees" in rec["fields"]
        ]
    except Exception as e:
        print(f"❌ Erreur récupération arrêts STAR : {e}")
        return []

def store_star_stops(stops):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("DELETE FROM star_stops")  # rafraîchir les arrêts

    for stop in stops:
        cursor.execute("""
            INSERT INTO star_stops (name, latitude, longitude)
            VALUES (?, ?, ?)
        """, (stop["name"], stop["lat"], stop["lon"]))

    conn.commit()
    conn.close()
    print(f"✅ {len(stops)} arrêts STAR enregistrés.\n")

# === COLLECTE DES DONNÉES TEMPS RÉEL STAR ===
def get_star_realtime():
    print("🚌 Collecte des passages STAR en temps réel...")
    try:
        response = requests.get(REALTIME_API_URL, params={
            "dataset": REALTIME_DATASET,
            "rows": 1000,
            "sort": "date"
        })
        return response.json().get("records", [])
    except Exception as e:
        print(f"❌ Erreur récupération STAR temps réel : {e}")
        return []

def store_star_realtime(records):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    ts = datetime.datetime.utcnow().isoformat()

    for rec in records:
        fields = rec.get("fields", {})
        stop = fields.get("nom_arret", "").strip()
        line = fields.get("nom_ligne", "inconnu")
        destination = fields.get("destination", "inconnue")
        status = fields.get("etat", "inconnu")
        sched = fields.get("arrivee_theorique", None)
        real = fields.get("arrivee_reelle", None)

        # Chercher les coords GPS dans la table des arrêts
        cursor.execute("SELECT latitude, longitude FROM star_stops WHERE name = ?", (stop,))
        result = cursor.fetchone()
        lat, lon = result if result else (None, None)

        cursor.execute("""
            INSERT INTO star_realtime (timestamp, line, destination, stop_name, latitude, longitude, status, arrival_scheduled, arrival_real)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (ts, line, destination, stop, lat, lon, status, sched, real))

    conn.commit()
    conn.close()
    print(f"✅ {len(records)} passages STAR enregistrés.\n")

# === DERNIÈRE ENTRÉE ===
def show_last_star():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM star_realtime ORDER BY id DESC LIMIT 1")
    row = cursor.fetchone()
    print("📊 Dernière entrée STAR :", row if row else "Aucune donnée.")
    conn.close()

# === BOUCLE PRINCIPALE ===
def main_loop():
    init_db()
    stops = get_star_stops()
    store_star_stops(stops)

    while True:
        records = get_star_realtime()
        store_star_realtime(records)
        show_last_star()
        print("⏳ Attente 5 minutes...\n")
        time.sleep(300)

if __name__ == "__main__":
    print("🚀 Script STAR complet lancé !\n")
    main_loop()
